# Parametric (spatial) building frame in reinforced concrete

Workshop N.2 - Nicola Sardella - 440648

<H1>[Struttura di riferimento](http://www.ingenio-web.it/immagini/CKEditor/Imm_2.jpg)</H1>
![alt text](Imm_2.jpg)

<H2>Dati contenuti nel file csv:</H2>
    

| A | B | C | D |DIST |ALT      | 
|---|---|---|---|-----|---------|
| 0 | 0 | 0 |   |     |         | 
|0.3|0.3|0.3|0.3|[4;4]|[2.5;2.5]|
| 3 | 0 | 0 |   |     |         | 
|0.3|0.3|0.3|0.3|[4;4]|[2.5;2.5]|

<h3>Librerie importate:</h3>

In [45]:
from pyplasm import *
import csv

<h3>Funzione per creare un telaio piano:</h3>

In [46]:
"""this function returns some HTC values, representative of a space frame. 
Input: 
bx,bz (given dimensions of beam section), 
px,py (given dimensions of pillar section), 
[dy1,dy2,...] (distances between axes of the pillars), 
[hz1,hz2,...] (interstory heights)"""
def spaceFrame(bx,bz,px,py,distances,heights):
    d=distances[0]*-1
    h=heights[0]
    if bx>px:
        dist=(bx-px)/2.
        pilX=[-dist,px]
        beaX=[bx]
    elif bx==px:
        beaX=[bx]
        pilX=[px]
    else:
        dist=(px-bx)/2.
        beaX=[-dist,bx]
        pilX=[px]
    pilY=[py]
    pilZ=[h]
    beaY=[py/2,0]
    beaZ=[h*-1,bz]
    for d in distances:
        pilY.append(d*-1);
        pilY.append(py)
        beaY.append((d)+py)
    beaY.append(0)
    #y.append(0)
    #y.append(py)
    for h in heights[1:]:
        pilZ.append(bz*-1)
        pilZ.append(h)
        beaZ.append(h*-1)
        beaZ.append(bz)
    beaY.append(py/2)
    pillarX=QUOTE(pilX)
    pillarY=QUOTE(pilY)
    pillarZ=QUOTE(pilZ)
    beamsX=QUOTE(beaX)
    beamsY=QUOTE(beaY)
    beamsZ=QUOTE (beaZ)
    beams2D=PROD([beamsX,beamsY])
    beams3D=PROD([beams2D,beamsZ])
    pillars2D=PROD([pillarX,pillarY])
    pillars3D=PROD([pillars2D,pillarZ])
    combine=STRUCT([beams3D,pillars3D])
    return combine

In [ ]:
<h3>Funzione per creare le travi orizzontali:</h3>

In [47]:
"""this function returns some HTC values, representative of a orizzontal beams. Its dimension (in WxLxD) is bzXpyXpx"""
def orizzontalBeams(bx,bz,py,px,distances,heights):
    beaX=[-px,bx]
    beaY=[]
    beaZ=[]
    for h in heights:
        beaZ.append(-h)
        beaZ.append(bz)
    for d in distances:
        beaY.append(py)
        beaY.append(-d)
    beaY.append(py)
    beamsX=QUOTE(beaX)
    beamsY=QUOTE(beaY)
    beamsZ=QUOTE (beaZ)
    beams2D=PROD([beamsX,beamsY])
    beams3D=PROD([beams2D,beamsZ])
    return beams3D

<h3>Funzione per leggere il file CSV e generare la struttura dell'edificio in cemento armato: </h3>

In [ ]:

"""this function returns a set of a space frame. 
Input: 
name (xxxxx in string of csv file named frame_data_xxxxx.csv)"""
def ggpl_bone_structure(fileName):
    r=None
    vettore=[]
    telaio=[]
    traviOrizz=[]
    struttura=[]
    struttura2=[]
    Vx=0
    with open(fileName) as csvfile:
        reader = csv.DictReader(csvfile)
        c=1
        i=0
        for row in reader:
            if c%2!=0 :#riga dispari
                Vx=eval(row['A'])
                Vy=eval(row['B'])
                Vz=eval(row['C'])
                vettore.append(T([1,2,3])([Vx,Vy,Vz]))
            else:#riga pari
                Bx=eval(row['A'])
                Bz=eval(row['B'])
                Px=eval(row['C'])
                Py=eval(row['D'])
                DIST=[]
                for elem in row['DIST'].replace("]","").replace("[","").split(";"):
                    DIST.append(eval(elem))
                ALT=[]
                for elem2 in row['ALT'].replace("]","").replace("[","").split(";"):
                    ALT.append(eval(elem2))
                d=spaceFrame(Bx,Bz,Px,Py,DIST,ALT)
                telaio.append(d)
                #costruisco le travi orizzontali per unire i telai
                traviOrizz.append(orizzontalBeams(Vx,Bz,Py,Px,DIST,ALT))
            c=c+1
            
    #Costruisco la serie di telai
    onetoten = range(0,len(telaio))
    for i in onetoten:
        struttura.append(vettore[i])
        struttura.append(telaio[i])
        if(i+1<len(telaio)):
            struttura2.append(vettore[i])
            struttura2.append(traviOrizz[i+1])
        
    a=STRUCT(struttura2)
    b=STRUCT(struttura)
    return STRUCT([a,b])
        
    

In [ ]:
VIEW(ggpl_bone_structure("frame_data_440648.CSV"))


<H3>Risultato ottenuto:</H3>
![alt text](Risultato.PNG)